In [21]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import os

# Get current working directory
current_dir = os.getcwd()
print(f"Current Working Directory: {current_dir}")

# Load the data
medal_counts = pd.read_csv('/Users/chris/MCM_2025_C/Data/summerOly_medal_counts.csv')
programs = pd.read_csv('/Users/chris/MCM_2025_C/Data/summerOly_programs.csv')
hosts = pd.read_csv('/Users/chris/MCM_2025_C/Data/summerOly_programs.csv')
3
# Define dependent and independent variables
dependent_var = 'Medal_Count'
independent_vars = ['Number_of_Athletes', 'Is_Host', 'Number_of_Sports', 'Total_Events']

# Prepare data
X = data[independent_vars]
X = sm.add_constant(X)
y = data[dependent_var]

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit Poisson regression
poisson_model = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
print(poisson_model.summary())

# Predict on test set
y_pred_poisson = poisson_model.predict(X_test)



Current Working Directory: /Users/chris/MCM_2025_C/Main


NameError: name 'data' is not defined

Adjust Predictions for Fixed Medal Counts:

In [ ]:
import scipy.stats as stats

# Assume 'events_list' contains all unique events
events_list = data['Event'].unique()
medal_types = ['Gold', 'Silver', 'Bronze']
final_medal_counts = pd.DataFrame(columns=['Country', 'Medal_Type', 'Medal_Count'])

for event in events_list:
    event_data = data[data['Event'] == event]
    for medal in medal_types:
        # Filter data for the current event
        event_medal_data = event_data[event_data['Medal_Type'] == medal]

        # Define X for this subset
        X_event = event_medal_data[independent_vars]
        X_event = sm.add_constant(X_event)

        # Predict expected counts
        expected_counts = poisson_model.predict(X_event)

        # Convert expected counts to probabilities
        probabilities = expected_counts / expected_counts.sum()

        # Handle cases where sum(expected_counts) is zero
        if expected_counts.sum() == 0:
            probabilities = np.ones(len(expected_counts)) / len(expected_counts)

        # Assign the medal based on probabilities
        assigned_country = np.random.choice(event_medal_data['Country'], size=1, p=probabilities)

        # Update final medal counts
        for country in assigned_country:
            final_medal_counts = final_medal_counts.append({'Country': country, 'Medal_Type': medal, 'Medal_Count': 1}, ignore_index=True)

# Aggregate medal counts per country
aggregated_medal_counts = final_medal_counts.groupby(['Country', 'Medal_Type']).sum().unstack(fill_value=0)
aggregated_medal_counts.columns = aggregated_medal_counts.columns.get_level_values(1)
aggregated_medal_counts['Total'] = aggregated_medal_counts.sum(axis=1)

# View the final medal counts
print(aggregated_medal_counts.head())


## Predicitons and Model Evaluation

In [ ]:
import scipy.stats as stats

# Assume 'events_list' contains all unique events
events_list = data['Event'].unique()
medal_types = ['Gold', 'Silver', 'Bronze']
final_medal_counts = pd.DataFrame(columns=['Country', 'Medal_Type', 'Medal_Count'])

for event in events_list:
    event_data = data[data['Event'] == event]
    for medal in medal_types:
        # Filter data for the current event
        event_medal_data = event_data[event_data['Medal_Type'] == medal]

        # Define X for this subset
        X_event = event_medal_data[independent_vars]
        X_event = sm.add_constant(X_event)

        # Predict expected counts
        expected_counts = poisson_model.predict(X_event)

        # Convert expected counts to probabilities
        probabilities = expected_counts / expected_counts.sum()

        # Handle cases where sum(expected_counts) is zero
        if expected_counts.sum() == 0:
            probabilities = np.ones(len(expected_counts)) / len(expected_counts)

        # Assign the medal based on probabilities
        assigned_country = np.random.choice(event_medal_data['Country'], size=1, p=probabilities)

        # Update final medal counts
        for country in assigned_country:
            final_medal_counts = final_medal_counts.append({'Country': country, 'Medal_Type': medal, 'Medal_Count': 1}, ignore_index=True)

# Aggregate medal counts per country
aggregated_medal_counts = final_medal_counts.groupby(['Country', 'Medal_Type']).sum().unstack(fill_value=0)
aggregated_medal_counts.columns = aggregated_medal_counts.columns.get_level_values(1)
aggregated_medal_counts['Total'] = aggregated_medal_counts.sum(axis=1)

# View the final medal counts
print(aggregated_medal_counts.head())


In [ ]:
num_simulations = 1000
simulation_results = []

for _ in range(num_simulations):
    temp_medal_counts = pd.DataFrame(columns=['Country', 'Medal_Type', 'Medal_Count'])
    for event in events_list:
        event_data = data[data['Event'] == event]
        for medal in medal_types:
            event_medal_data = event_data[event_data['Medal_Type'] == medal]
            X_event = event_medal_data[independent_vars]
            X_event = sm.add_constant(X_event)
            expected_counts = poisson_model.predict(X_event)
            probabilities = expected_counts / expected_counts.sum() if expected_counts.sum() > 0 else np.ones(len(expected_counts)) / len(expected_counts)
            assigned_country = np.random.choice(event_medal_data['Country'], size=1, p=probabilities)
            for country in assigned_country:
                temp_medal_counts = temp_medal_counts.append({'Country': country, 'Medal_Type': medal, 'Medal_Count': 1}, ignore_index=True)
    aggregated_temp = temp_medal_counts.groupby(['Country', 'Medal_Type']).sum().unstack(fill_value=0)
    aggregated_temp.columns = aggregated_temp.columns.get_level_values(1)
    aggregated_temp['Total'] = aggregated_temp.sum(axis=1)
    simulation_results.append(aggregated_temp['Total'])

# Convert to DataFrame
simulation_df = pd.DataFrame(simulation_results)

# Calculate prediction intervals
prediction_intervals = simulation_df.quantile([0.025, 0.5, 0.975]).T
prediction_intervals.columns = ['Lower_95%', 'Median', 'Upper_95%']

# Merge with final counts
final_with_intervals = aggregated_medal_counts.join(prediction_intervals)

print(final_with_intervals.head())
